In [72]:
from collections import Counter
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import numpy as np
import plotly
from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import gensim
from textblob import TextBlob
# plotly configuration
plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [82]:
H = pd.read_csv('F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Xiaomi_final_brand.csv',encoding='utf-8',error_bad_lines=False,engine='python')
#H = pd.read_csv('F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Xiaomi_final_brand.csv',encoding='utf-8',error_bad_lines=False,engine='python')

#,usecols=['text']

In [83]:
H

,id,date,text,permalink,Brand
0,925483153032179712,2017-10-31 23:02,Xiaomi Mi MIX 2 review: A phone for performanc...,https://twitter.com/pridenewsza/status/9254831...,Xiaomi
1,925460014449401856,2017-10-31 21:30,Xiaomi is like Muslim of phones.. Every month ...,https://twitter.com/akramismm/status/925460014...,Xiaomi
2,925424043699908608,2017-10-31 19:07,Got these two phones to review from Intertec. ...,https://twitter.com/iloveqatar/status/92542404...,Xiaomi
3,925354189936402432,2017-10-31 14:30,Xiaomi ships 10 million phones in October: Xia...,https://twitter.com/UniverSmartphon/status/925...,Xiaomi
4,925347723167043584,2017-10-31 14:04,http://Israelopadeyi.com - Wealth And Tech Dig...,https://twitter.com/israelopadeyi/status/92534...,Xiaomi
5,925329922989604864,2017-10-31 12:53,"Still, by far the most beautiful phone I’ve ev...",https://twitter.com/AJHamadi/status/9253299229...,Xiaomi
6,925319286649896961,2017-10-31 12:11,And the much awaited # Nokia2 is finally here....,https://twitter.com/Zayvk/status/9253192866498...,Xiaomi
7,925316294471241728,2017-10-31 11:59,"Nokia 2 Vs Xiaomi Redmi 4A: Price, specificati...",https://twitter.com/Android_Updates/status/925...,Xiaomi
8,925301040345419776,2017-10-31 10:58,And the hunt continues for an Upper Mid-Range ...,https://twitter.com/AshRock22/status/925301040...,Xiaomi
9,925283583027113984,2017-10-31 09:49,I challenge you to find LTPS And LPDDR3 In xia...,https://twitter.com/RSS_Haters_LIE/status/9252...,Xiaomi


In [75]:
#H['text'] =  str(H['text'])

In [86]:
#tweet = H.text[10000]
H['id'] = H['id']
H['date'] = H['date']
H['text'] = H['text']
H['permalink'] = H['permalink']
H['Brand'] = H['Brand']
H['polarity'] = H['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
H['subjectivity'] = H['text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
#blob = TextBlob(tweet)

In [87]:
H

,id,date,text,permalink,Brand,brand,polarity,subjectivity
0,925483153032179712,2017-10-31 23:02,Xiaomi Mi MIX 2 review: A phone for performanc...,https://twitter.com/pridenewsza/status/9254831...,Xiaomi,Xiaomi,0.000000,0.000000
1,925460014449401856,2017-10-31 21:30,Xiaomi is like Muslim of phones.. Every month ...,https://twitter.com/akramismm/status/925460014...,Xiaomi,Xiaomi,0.136364,0.454545
2,925424043699908608,2017-10-31 19:07,Got these two phones to review from Intertec. ...,https://twitter.com/iloveqatar/status/92542404...,Xiaomi,Xiaomi,0.000000,0.000000
3,925354189936402432,2017-10-31 14:30,Xiaomi ships 10 million phones in October: Xia...,https://twitter.com/UniverSmartphon/status/925...,Xiaomi,Xiaomi,0.000000,0.000000
4,925347723167043584,2017-10-31 14:04,http://Israelopadeyi.com - Wealth And Tech Dig...,https://twitter.com/israelopadeyi/status/92534...,Xiaomi,Xiaomi,0.000000,0.000000
5,925329922989604864,2017-10-31 12:53,"Still, by far the most beautiful phone I’ve ev...",https://twitter.com/AJHamadi/status/9253299229...,Xiaomi,Xiaomi,0.483333,0.833333
6,925319286649896961,2017-10-31 12:11,And the much awaited # Nokia2 is finally here....,https://twitter.com/Zayvk/status/9253192866498...,Xiaomi,Xiaomi,0.225000,0.525000
7,925316294471241728,2017-10-31 11:59,"Nokia 2 Vs Xiaomi Redmi 4A: Price, specificati...",https://twitter.com/Android_Updates/status/925...,Xiaomi,Xiaomi,0.000000,0.000000
8,925301040345419776,2017-10-31 10:58,And the hunt continues for an Upper Mid-Range ...,https://twitter.com/AshRock22/status/925301040...,Xiaomi,Xiaomi,0.000000,0.000000
9,925283583027113984,2017-10-31 09:49,I challenge you to find LTPS And LPDDR3 In xia...,https://twitter.com/RSS_Haters_LIE/status/9252...,Xiaomi,Xiaomi,-0.083333,0.233333


In [88]:
H.to_csv('F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Xiaomi_final_brand_annotated.csv',encoding='utf-8',index=False)


In [46]:
#H['Polarity']
for tweet in H:
    tweet['polarity'] = float(tweet['TextBlob'].sentiment.polarity)
    tweet['subjectivity'] = float(tweet['TextBlob'].sentiment.subjectivity)

    if tweet['polarity'] >= 0.1:
        tweet['sentiment'] = 'positive'
    elif tweet['polarity'] <= -0.1:
        tweet['sentiment'] = 'negative'
    else:
        tweet['sentiment'] = 'neutral'

TypeError: string indices must be integers

In [19]:
# try...............return list of Sentence objects
blob.sentences

[Sentence("Someone with an Huawei P8 lite."),
 Sentence("I need help with something."),
 Sentence("I keep downloading a document on my phone."),
 Sentence("But this keeps showing up."),
 Sentence("And I can't seem to find the document anywhere."),
 Sentence("pic.twitter.com/TubmgSs4Ir")]

In [20]:
#try............getting words
blob.words

WordList(['Someone', 'with', 'an', 'Huawei', 'P8', 'lite', 'I', 'need', 'help', 'with', 'something', 'I', 'keep', 'downloading', 'a', 'document', 'on', 'my', 'phone', 'But', 'this', 'keeps', 'showing', 'up', 'And', 'I', 'ca', "n't", 'seem', 'to', 'find', 'the', 'document', 'anywhere', 'pic.twitter.com/TubmgSs4Ir'])

In [21]:
#try...............getting count of words
blob.word_counts

defaultdict(int,
            {'a': 1,
             'an': 1,
             'and': 1,
             'anywhere': 1,
             'but': 1,
             'ca': 1,
             'document': 2,
             'downloading': 1,
             'find': 1,
             'help': 1,
             'huawei': 1,
             'i': 3,
             'keep': 1,
             'keeps': 1,
             'lite': 1,
             'my': 1,
             "n't": 1,
             'need': 1,
             'on': 1,
             'p8': 1,
             'phone': 1,
             'pic.twitter.com/tubmgss4ir': 1,
             'seem': 1,
             'showing': 1,
             'someone': 1,
             'something': 1,
             'the': 1,
             'this': 1,
             'to': 1,
             'up': 1,
             'with': 2})

In [22]:
#
blob.tags

[('Someone', 'NN'),
 ('with', 'IN'),
 ('an', 'DT'),
 ('Huawei', 'NNP'),
 ('P8', 'NNP'),
 ('lite', 'NN'),
 ('I', 'PRP'),
 ('need', 'VBP'),
 ('help', 'NN'),
 ('with', 'IN'),
 ('something', 'NN'),
 ('I', 'PRP'),
 ('keep', 'VBP'),
 ('downloading', 'VBG'),
 ('a', 'DT'),
 ('document', 'NN'),
 ('on', 'IN'),
 ('my', 'PRP$'),
 ('phone', 'NN'),
 ('But', 'CC'),
 ('this', 'DT'),
 ('keeps', 'VBZ'),
 ('showing', 'VBG'),
 ('up', 'RP'),
 ('And', 'CC'),
 ('I', 'PRP'),
 ('ca', 'MD'),
 ("n't", 'RB'),
 ('seem', 'VB'),
 ('to', 'TO'),
 ('find', 'VB'),
 ('the', 'DT'),
 ('document', 'NN'),
 ('anywhere', 'RB'),
 ('pic.twitter.com/TubmgSs4Ir', 'NN')]

In [23]:
blob.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [27]:
for row in H:
        #sentence = row[0]
        #blob = TextBlob(sentence)
        #print blob.sentiment
        print(row)
        print('-------------------------------------')
#for sentence in blob.sentences:
 #   print(sentence.sentiment.polarity)

text
-------------------------------------


In [26]:
import csv
from textblob import TextBlob
H = pd.read_csv('F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Iphone_final_brand.csv',encoding='utf-8',error_bad_lines=False,engine='python')


In [27]:
infile = 'F:/web mining/webmining project/crawler/twitter_Web_Mining/Data/Huawei_final_brand.csv'

with open(infile, 'r') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        #sentence = row[0]
        #blob = TextBlob(sentence)
        #print blob.sentiment
        print(row[2])
        print('-------------------------------------')

text
-------------------------------------
No. I has serious problems with call quality and had to deal with Huawei instead of Google for my Nexus phone. A disaster.
-------------------------------------
Teardown of Huawei Mate 10 - the 2nd or 3rd biggest phone maker, depending on Q. Samsung wins NAND and Micron DRAM: http://techinsights.com/about-techinsights/overview/blog/huawei-mate-10-teardown/ â€¦
-------------------------------------
I didnÂ´t see any differences between this phone and say, a Huawei. The only difference I appreciate is that it is a lot more expensive
-------------------------------------
@ Google @ HuaweiMobile @ Nexus6pIssues updated for today. Phone shut off and when I turned back on this is how much battery I had. New record. pic.twitter.com/pEywbedmYv
-------------------------------------
Tomorrow on IG, 3pm. Tell me why you are SUPERIOR and win a Huawei Y5 phone # GoSuperiorhttps://twitter.com/caroline4real/status/919954489142112256 â€¦
---------------------

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3877: character maps to <undefined>

In [ ]:
for row in rows:
    sentence = row[0]
    blob = TextBlob(sentence)
    print blob.sentiment

In [6]:
from textblob import TextBlob

In [9]:
text = '''The titular threat of The Blob has always struck me as the ultimate movie
monster: an insatiably hungry, amoeba-like mass able to penetrate
virtually any safeguard, capable of--as a doomed doctor chillingly
describes it--"assimilating flesh on contact.
Snide comparisons to gelatin be damned, it's a concept with the most
devastating of potential consequences, not unlike the grey goo scenario
proposed by technological theorists fearful of
artificial intelligence run rampant.
'''

In [10]:
text

'The titular threat of The Blob has always struck me as the ultimate movie\nmonster: an insatiably hungry, amoeba-like mass able to penetrate\nvirtually any safeguard, capable of--as a doomed doctor chillingly\ndescribes it--"assimilating flesh on contact.\nSnide comparisons to gelatin be damned, it\'s a concept with the most\ndevastating of potential consequences, not unlike the grey goo scenario\nproposed by technological theorists fearful of\nartificial intelligence run rampant.\n'

In [11]:
blob = TextBlob(text)

In [12]:
blob.tags

[('The', 'DT'),
 ('titular', 'JJ'),
 ('threat', 'NN'),
 ('of', 'IN'),
 ('The', 'DT'),
 ('Blob', 'NNP'),
 ('has', 'VBZ'),
 ('always', 'RB'),
 ('struck', 'VBN'),
 ('me', 'PRP'),
 ('as', 'IN'),
 ('the', 'DT'),
 ('ultimate', 'JJ'),
 ('movie', 'NN'),
 ('monster', 'NN'),
 ('an', 'DT'),
 ('insatiably', 'RB'),
 ('hungry', 'JJ'),
 ('amoeba-like', 'JJ'),
 ('mass', 'NN'),
 ('able', 'JJ'),
 ('to', 'TO'),
 ('penetrate', 'VB'),
 ('virtually', 'RB'),
 ('any', 'DT'),
 ('safeguard', 'NN'),
 ('capable', 'JJ'),
 ('of', 'IN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('doomed', 'JJ'),
 ('doctor', 'NN'),
 ('chillingly', 'RB'),
 ('describes', 'VBZ'),
 ('it', 'PRP'),
 ('assimilating', 'VBG'),
 ('flesh', 'NN'),
 ('on', 'IN'),
 ('contact', 'NN'),
 ('Snide', 'JJ'),
 ('comparisons', 'NNS'),
 ('to', 'TO'),
 ('gelatin', 'VB'),
 ('be', 'VB'),
 ('damned', 'VBN'),
 ('it', 'PRP'),
 ("'s", 'VBZ'),
 ('a', 'DT'),
 ('concept', 'NN'),
 ('with', 'IN'),
 ('the', 'DT'),
 ('most', 'RBS'),
 ('devastating', 'JJ'),
 ('of', 'IN'),
 ('potenti